# GRPM MeSH Querying 

This notebook is engineered to screen the previously retrieved genetic polymorphism data using selected MeSH terms. It works with MeSH sets that are used as hooks to retrieve subsets of genes and polymorphisms from the "GRPM" dataset.

# Import Packages

In [1]:
import importlib
import subprocess

try:
    importlib.import_module('pygrpm')
except ImportError:
    subprocess.check_call(["pip", "install", "git+https://github.com/johndef64/GRPM_system.git"])

from pygrpm import *

import os
import glob
import random
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt


# Get requirements

In [3]:
# Get GRPM Dataset from Zenodo Repository
#https://zenodo.org/record/8205724  DOI: 10.5281/zenodo.8205724

if not os.path.exists("grpm_dataset/grpm_dataset.parquet"):
    timea = datetime.now()
    get_and_extract('grpm_dataset')
    print('Download and extraction time ',datetime.now()-timea)

if not os.path.exists("ref-mesh/"):
    timea = datetime.now()
    get_and_extract('ref-mesh')
    get_topic_terms()
    print('Download and extraction time ',datetime.now()-timea)

## Import GRPM dataset [required]

In [12]:
#Load GRPM db Report-----------------------------------------
import pyarrow
db_path = 'grpm_dataset/'

# choose gene type:
gene_types = {'PCG' : "protein coding genes",
              'RNA' : "rna genes",
              'PSEUDO' : "pseudo genes"}
gene_type = 'PCG'


#get gene list from grpm report
GRPM_report = pd.read_parquet(db_path+'/grpm_report.parquet', engine = 'pyarrow').reset_index().rename(columns={'index':'gene'})
cols = ['gene', 'rsid', 'pmid', 'mesh']
GRPM_report = GRPM_report[GRPM_report.type == gene_type].reset_index(drop=True)
grpm_genes_list = GRPM_report.gene.to_list()

#Import GRPM dataset-------------------------------------------
print('importing GRPM Dataset...')
time_load_1 = datetime.now()
pcg_grpm, rna_grpm, pseudo_grpm = grpm_importer()
grpm_dataset = pcg_grpm
db_tag  = gene_type.lower()

#grpm_dataset['pmid'] = grpm_dataset['pmid'].astype(str) #convert pmid type in str
time_load_2 = datetime.now()
print('time load:',time_load_2-time_load_1)

importing GRPM Dataset...
Importing time:  0:00:07.185983
pcg: 776.19 MB
rna: 58.18 MB
pseudo: 1.93 MB
time load: 0:00:13.193017


# Query GRPM Dataset

## Set directory & import data
Check available ref-MeSH lists
    - gene list
    - survey directory
    - ref-mesh list

In [13]:
#Check avalable refs:
ref_path = "ref-mesh/"  # Replace with the actual ref mesh path

#---------------------------------
#use random mesh list?
random_mesh = False
if random_mesh: ref_path = "ref-mesh/random_lists/"
#---------------------------------

# Create a file path pattern to match CSV files
file_pattern = os.path.join(ref_path, "*.csv")

# Use glob to get a list of file paths matching the pattern
csv_files = glob.glob(file_pattern)

csv_files_name = []
# Print the list of CSV files
for file in csv_files:
    file_name = os.path.basename(file)
    csv_files_name.append(file_name)

pd.set_option('display.max_rows', 100)
print('Available reference mesh lists:')
csv_files_df = pd.Series(csv_files_name)

csv_file_tag = pd.DataFrame()
if not random_mesh:
    csv_file_tag = csv_files_df.str.extract(r'ref_mesh_(.*)\.csv', expand=False).dropna().reset_index(drop=True)
else:
    csv_file_tag = csv_files_df.str.extract(r'(.*)\.csv', expand=False).dropna().reset_index(drop=True)


#------------------------------------------------------
# define directory folder path:
survey_path = 'grpm_surveys/' # keep default to use root path

# choose ref_mesh.csv tab:
topic_tag   = csv_file_tag[int(input('\Select index from available ref-mesh list:\n'+str(csv_file_tag)))]
add         = ''    # additional survey directory tag
#------------------------------------------------------


# (1) Create survey directory:
survey_path = survey_path+'grpm_random/' if random_mesh else survey_path
directory = survey_path + 'grpm_survey_' + db_tag + '_' + topic_tag + add
if not os.path.exists(directory):
    os.makedirs(directory)


# (2) Import Mesh-reference list:
ref_filename = "ref_mesh_" + topic_tag + ".csv" if not random_mesh else topic_tag + ".csv"
ref = pd.read_csv(ref_path + ref_filename, index_col=0)

if 'mesh' not in ref.columns:
    ref = ref.rename(columns={'Preferred Label': 'mesh'})

ref_mesh_n = ref.mesh.nunique()
ref_mesh_list = ref['mesh'].drop_duplicates()

#----------------------------------------------------------
print('\n', ref_mesh_list)

Available reference mesh lists:

 0                                 Stearoyl-CoA Desaturase
4            Plasma Membrane Calcium-Transporting ATPases
8              Glycine Plasma Membrane Transport Proteins
12           Serotonin Plasma Membrane Transport Proteins
16      Norepinephrine Plasma Membrane Transport Proteins
                              ...                        
1570                            Neoplasms, Adipose Tissue
1571                                       Adipose Tissue
1575                                Adipose Tissue, Beige
1576                                Adipose Tissue, White
1577                                Adipose Tissue, Brown
Name: mesh, Length: 528, dtype: object


## Get GRPM dataset statistics [required]


In [14]:
%time
# GET GRPM dataset Stats
time_1 = datetime.now()
grpm_dataset['pmidmesh'] = grpm_dataset['pmid'] + grpm_dataset['mesh']
print(datetime.now() -time_1)
grpm_dataset.head()

time_1 = datetime.now()
grpm_gene_stats = grpm_dataset[['gene', 'rsid', 'pmid', 'mesh', 'pmidmesh']].groupby('gene').describe(include='all')
print(datetime.now() -time_1)
grpm_gene_stats.head()

CPU times: total: 0 ns
Wall time: 0 ns
0:00:02.537021
0:01:06.114094


rsid                           pmid                        mesh  \
        count unique          top freq count unique       top freq count   
gene                                                                       
A1BG      118      4     rs893184   68   118      7  23690342   40   118   
A1CF      362      7   rs10821905  185   362     23  29437585   31   362   
A2M      1160     21        rs669  676  1160     57  32747830   92  1160   
A2ML1    1065     32  rs863224951  119  1065     13  31009165  575  1065   
A3GALT2    19      1  rs376200069   19    19      1  28506304   19    19   

                                    pmidmesh         \
        unique             top freq    count unique   
gene                                                  
A1BG        75    Hypertension    8      118     94   
A1CF       137          Humans   22      362    318   
A2M        365          Humans   61     1160    887   
A2ML1      117    Otitis Media   51     1065    227   
A3GALT2     11  Adenocarcinoma    3       19     11   

                                             
                                   top freq  
gene                                         
A1BG                     32279138HIV-1    3  
A1CF      30529582Colorectal Neoplasms    4  
A2M      24011543Amyloid beta-Peptides    8  
A2ML1               31009165Adolescent   23  
A3GALT2         28506304Adenocarcinoma    3

## Execute MeSH Query

Python Query

In [21]:
ref_mesh_tuple = tuple(ref_mesh_list)
random.sample(ref_mesh_tuple, 10)

# Query Example
# ref_mesh_tuple = tuple(['Sodium-Coupled Vitamin C Transporters',
#                         'Micronutrients',
#                         'Biotinidase Deficiency',
#                         'Lipid Metabolism Disorders',
#                         'Vitamin D-Binding Protein',
#                         'Pyridoxal Phosphate',
#                         'Choline-Phosphate Cytidylyltransferase',
#                         'Vitamin D3 24-Hydroxylase'])

['Chorea',
 'Multiple Sclerosis, Chronic Progressive',
 'Serotonin and Noradrenaline Reuptake Inhibitors',
 'Receptors, Histamine H2',
 'Long-Term Potentiation',
 'GABA-A Receptor Antagonists',
 'Brain Mapping',
 'Nitric Oxide Synthase Type I',
 'Cerebellar Diseases',
 'Oligodendroglia']

In [15]:
%time
# Preprocessing: 
time_1 = datetime.now()
genes = grpm_dataset.gene.drop_duplicates().to_list()

# Execute the query using pandas
def query_dataset(ds, my_tuple, field = 'mesh'):
    ds = ds[ds[field].isin(my_tuple)].drop_duplicates()
    return ds

ref_mesh_tuple = tuple(ref_mesh_list)
result = query_dataset(grpm_dataset, ref_mesh_tuple, field = 'mesh') 
print(datetime.now() -time_1)

# save Query Output
grpm_match_full = result
result.sample(10)

CPU times: total: 0 ns
Wall time: 0 ns
0:00:01.891008


,gene,type,rsid,pmid,mesh,qualifier,major,pmidmesh
7951660,MTNR1B,PCG,rs61747139,20200315,"Diabetes Mellitus, Type 2",genetics,True,"20200315Diabetes Mellitus, Type 2"
12772488,POLG,PCG,rs113994098,31147703,Mitochondrial Diseases,genetics,True,31147703Mitochondrial Diseases
408211,TNF,PCG,rs1800629,33589701,Metabolic Syndrome,etiology,True,33589701Metabolic Syndrome
5258004,ICAM4,PCG,rs281437,28095483,Diabetes Mellitus,genetics,True,28095483Diabetes Mellitus
16193286,PLA2G2A,PCG,rs876018,27608594,Metabolic Syndrome,genetics,True,27608594Metabolic Syndrome
11767953,PHACTR1,PCG,rs12526453,27612170,Lipase,genetics,True,27612170Lipase
12685442,JAK3,PCG,rs193922364,23384681,Cell Proliferation,None,False,23384681Cell Proliferation
16270710,PCSK9,PCG,rs11591147,30445611,Cholesterol,blood,False,30445611Cholesterol
15423196,PRKCZ,PCG,rs2503706,18615853,"Diabetes Mellitus, Type 2",genetics,True,"18615853Diabetes Mellitus, Type 2"
12186536,PPARG,PCG,rs121909244,32852625,Hypertension,None,True,32852625Hypertension


In [ ]:
# SAVE
if simple_bool('save output?'): 
    grpm_match_full[['gene', 'rsid', 'pmid', 'mesh']].to_csv(directory+'/grpmx_filtered_output.csv')

# Extraction Data Report

## Get GRPM Subset Statistics [required]

In [61]:
%time

grpm_match_full['pmidmesh'] = grpm_match_full['pmid']+grpm_match_full['mesh']
time_1 = datetime.now()
grpm_match_gene_stats = grpm_match_full[['gene', 'rsid', 'pmid', 'mesh', 'pmidmesh']].groupby('gene').describe(include='all')
print(datetime.now() -time_1)
grpm_match_gene_stats.head()

CPU times: total: 0 ns
Wall time: 0 ns
0:00:50.933666


rsid                           pmid                        mesh  \
       count unique          top freq count unique       top freq count   
gene                                                                      
A1BG       2      1     rs893184    2     2      1  23690342    2     2   
A1CF       5      2   rs10821905    3     5      3  30014513    3     5   
A2M      137     11        rs669   98   137     41  24011543   16   137   
A4GALT    11      6  rs756992490    3    11      4  22241068    3    11   
A4GNT      1      1    rs2246945    1     1      1  21811255    1     1   

                                      pmidmesh         \
       unique                top freq    count unique   
gene                                                    
A1BG        1  Membrane Proteins    2        2      1   
A1CF        2  Parkinson Disease    3        5      3   
A2M        29  Alzheimer Disease   46      137     71   
A4GALT      6    Protein Folding    3       11      6   
A4GNT       1   Nuclear Proteins    1        1      1   

                                            
                                  top freq  
gene                                        
A1BG        23690342Membrane Proteins    2  
A1CF        30014513Parkinson Disease    3  
A2M     24011543Amyloid beta-Peptides    8  
A4GALT        22241068Protein Folding    3  
A4GNT        21811255Nuclear Proteins    1

## Other Stats [optional]

In [ ]:
time_1 = datetime.now()
grpm_match_rsid_stats = grpm_match_full[['rsid', 'pmid']].groupby('rsid').describe(include='all')#agg(lambda x: x.unique())
print(datetime.now() -time_1)
#grpm_match_rsid_stats.head()
grpm_match_rsid_stats_sort = grpm_match_rsid_stats.sort_values(by=('pmid','unique'), ascending= False)
top10rsid = list(grpm_match_rsid_stats_sort[:10].index)
top10rsid

In [ ]:
time_1 = datetime.now()
grpm_match_mesh_stats = grpm_match_full[['pmid', 'mesh']].groupby('mesh').describe(include='all')#agg(lambda x: x.unique())
print(datetime.now() -time_1)
#grpm_match_mesh_stats.head()

grpm_match_mesh_stats_sort = grpm_match_mesh_stats.sort_values(by=('pmid','unique'), ascending= False)
top10mesh = list(grpm_match_mesh_stats_sort[:10].index)
top10mesh

In [ ]:
time_1 = datetime.now()
grpm_match_pmids_stats = grpm_match_full[['gene', 'rsid', 'pmid', 'mesh']].groupby('pmid').describe(include='all')#agg(lambda x: x.unique())
print(datetime.now() -time_1)
grpm_match_pmids_stats.head()

## Compile Survey Report 
Gather Extraction Data
(approx. 10 minutes required)

In [28]:
from tqdm import tqdm

#---------------------------------------------
# Edit saving options:
checkpoint = 10 # write Report each "n" genes

include_top10 = True # for a faster job
save_plot = False  # only if include_top10 = True

run_sample = simple_bool('Run a sample?') # set True just to run a test
sample_size = 100

df_report_complete = pd.DataFrame()
#---------------------------------------------

# define gene list
import random
if run_sample:
    genes = random.sample(grpm_genes_list[:], sample_size)
else:
    genes = grpm_genes_list

# Start cycle
time_start = datetime.now()
for gene in tqdm(genes):

    time_alpha = datetime.now()
    timestamp = time_alpha.strftime('%Y%m%d%H%M%S')

    # Pre-Selection Metrics ===========
    grpm_gene_stats_gene = grpm_gene_stats.loc[gene]
    starting_pmid        =  grpm_gene_stats_gene.pmid.loc['unique']
    starting_mesh        =  grpm_gene_stats_gene.mesh.loc['unique']
    lit1_rsid            =  grpm_gene_stats_gene.rsid.loc['unique']
    starting_pmidmesh    =  grpm_gene_stats_gene.pmidmesh.loc['unique']
    
    #  Post-Selection Metrics ===========
    if gene in grpm_match_gene_stats.index:
        grpm_match_gene_stats_gene = grpm_match_gene_stats.loc[gene]
        matching_rsid                = grpm_match_gene_stats_gene.rsid.loc['unique']
        dropped_rsid                 = lit1_rsid - matching_rsid
        matching_pmids               = grpm_match_gene_stats_gene.pmid.loc['unique']
        matching_mesh                = grpm_match_gene_stats_gene.mesh.loc['unique']
        matching_pmidmesh            = grpm_match_gene_stats_gene.pmidmesh.loc['unique']
    else:
        matching_rsid                = 0
        dropped_rsid                 = lit1_rsid - 0
        matching_pmids               = 0
        matching_mesh                = 0
        matching_pmidmesh            = 0
                

    #------------------------
    if not include_top10:
        matching_rsid_pmid10  = 'na'
        matching_rsid_pmid100 = 'na'
        top10rsid             = 'na'
        top10mesh             = 'na'
    else:
        #=====================================
        # Get gene_grpm (slow step)
        grpm_gene = grpm_match_full.loc[grpm_match_full['gene'] == gene]
        #print(datetime.now() -time_alpha)
    
        dfmatch_full = grpm_gene
        dfmatch_less = dfmatch_full[['pmid', 'rsid', 'mesh']].drop_duplicates()
        #=====================================

        #=====================================
        ## 1. groupby.describe analysis by [rsid]
        dfmatch_less_rsid = dfmatch_less.groupby('rsid').describe().reset_index()
        dfmatch_less_rsid.columns = dfmatch_less_rsid.columns.to_flat_index()
        new_column_names = ['rsid', 'pmid-count', 'pmid-unique','pmid-top','pmid-freq','mesh-count', 'mesh-unique','mesh-top','mesh-freq']
        dfmatch_less_rsid.columns = new_column_names

        ### statistics:
        #grpm_match_rsid_stats_gene = grpm_match_rsid_stats.loc[gene]
        matching_rsid_pmid10 = len(dfmatch_less_rsid[dfmatch_less_rsid['pmid-unique']>10])
        matching_rsid_pmid100 = len(dfmatch_less_rsid[dfmatch_less_rsid['pmid-unique']>100])

        ### sorting, top10
        dfmatch_less_rsidless = dfmatch_less_rsid[['rsid','pmid-unique','mesh-unique']]
        dfmatch_less_rsidlesssort = dfmatch_less_rsidless.sort_values(by='pmid-unique', ascending= False).reset_index(drop=True)
        top10rsid = dfmatch_less_rsidlesssort['rsid'][:10].tolist()
        #==================================
        
        #==================================
        ## 2. groupby.describe analysis by [mesh]
        dfmatch_less_mesh = dfmatch_less.groupby('mesh').describe().reset_index()
        dfmatch_less_mesh.columns = dfmatch_less_mesh.columns.to_flat_index()
        # simplify df.groupby.describe, convert Multicolumn to single column 
        new_column_names = ['mesh', 'pmid-count', 'pmid-unique','pmid-top','pmid-freq','rsid-count', 'rsid-unique','rsid-top','rsid-freq']
        dfmatch_less_mesh.columns = new_column_names
        dfmatch_less_mesh_less = dfmatch_less_mesh[['mesh','pmid-unique','rsid-unique']]
        
        ### add frequency, top10
        samplepmid_count = len(dfmatch_less.pmid.drop_duplicates())
        dfmatch_less_mesh_less_frq = dfmatch_less_mesh_less.copy()
        mesh_frq = dfmatch_less_mesh_less_frq.loc[:,'pmid-unique'].astype(float)/samplepmid_count
        dfmatch_less_mesh_less_frq.loc[:,'mesh frequency'] = round(mesh_frq,3)#*100
        dfmatch_less_mesh_less_frqsort = dfmatch_less_mesh_less_frq.sort_values(by='pmid-unique',ascending=False).reset_index(drop=True)
        top10mesh = dfmatch_less_mesh_less_frqsort['mesh'][:10].tolist()
        #==================================

        if save_plot:
            # create a scatter plot
            x = dfmatch_less_mesh_less_frqsort['mesh'].head(30)
            y = dfmatch_less_mesh_less_frqsort['pmid-unique'].head(30)
            plt.figure(figsize=(5, 8))
            plt.title('Scatter Plot: '+gene+' pmid-mesh (filtered)', loc='center',pad=10)
            plt.scatter(y, x)
            plt.gca().invert_yaxis()
            #plt.subplots_adjust(left=0.3, right=0.9, bottom=0.3, top=0.9)
            #plt.xticks(rotation=90)
            plt.tick_params(axis='x', which='both', top=True, bottom=False, labeltop=True, labelbottom=False)
            plt.xlabel('pmid count', position=(0.5, 1.08))
            ax = plt.gca()
            ax.xaxis.set_label_position('top')
            #plt.show()
            plt.savefig(directory+'/'+gene+'_mesh_plot_'+timestamp+'_filtered.png',dpi=120, bbox_inches = "tight")
            plt.close()
        else:
            pass


    # Collect REPORT data ==================================

    report = { 'reference_mesh': ref_mesh_n,
               'starting_pmidmesh': starting_pmidmesh,
               'starting_pmid' : starting_pmid,
               'starting_mesh': starting_mesh,
               'starting_rsid': lit1_rsid,
               'matching_pmidmesh': matching_pmidmesh,
               'matching_pmids': matching_pmids,
               'matching_mesh': matching_mesh,
               'matching_rsid': matching_rsid,
               'dropped_rsid': dropped_rsid,
               'matching_mesh_ratio': round((matching_mesh/starting_mesh),3),
               'matching_pmids_ratio': round((matching_pmids/starting_pmid),3),
               'matching_pmidmesh_ratio':  round((matching_pmidmesh/starting_pmidmesh),3),
               'matching_rsid_ratio': round((matching_rsid/lit1_rsid),3),
               'matching_rsid_pmid10': matching_rsid_pmid10,
               'matching_rsid_pmid100': matching_rsid_pmid100,
               'matching_top10mesh':str(top10mesh),
               'matching_top10rsid':str(top10rsid),
               }
    
    # UPDATE REPORT ------------
    df_new_raw = pd.DataFrame(report, index=[gene])
    df_report_complete = pd.concat([df_report_complete, df_new_raw])

    full_runtime = datetime.now() - time_alpha
    #print((gene+'_runtime:').ljust(18)+ str(full_runtime).ljust(15), ' Genes processed:', genes.index(gene), 'on', len(genes))
    total_seconds = full_runtime.total_seconds()

    # save checkpoint----------------------
    if genes.index(gene) > 1 and genes.index(gene) % checkpoint == 0:
        df_report_complete.to_csv(directory+'/GRPMX_report.csv')
        #print("saved checkpoint")
    else:
        pass
    #==================================


# Save report csv  (saving translate version is a code atavism)
df_report_complete.to_csv(directory+'/GRPMX_report.csv')

"""
# #Update gene values (remove previous gene entry)
GRPMX_report = pd.read_csv(directory+'/GRPMX_report.csv', index_col=0)
time_load_1 = datetime.now()
for gene in grpm_genes_list:
    if gene+'.1' in GRPMX_report.columns:
        GRPMX_report = GRPMX_report.drop(columns = gene)
        GRPMX_report = GRPMX_report.rename(columns={gene+'.1': gene})
    else:
        pass
    
print(datetime.now() - time_load_1)
GRPMX_report.to_csv(directory+'/GRPMX_report.csv')
"""

time_finish = datetime.now()
time_batch = time_finish - time_start

if os.path.isfile('run_time.txt'):
    with open('run_time.txt', 'a') as file:
        file.write(topic_tag+':\n\ttime batch: '+str(time_batch)+'\n\truntime/gene: '+str(time_batch/len(genes))+'\n\n')
else:
    with open('run_time.txt', 'w') as file:
        file.write(topic_tag+':\n\ttime batch: '+str(time_batch)+'\n\truntime/gene: '+str(time_batch/len(genes))+'\n\n')

print('time batch:',time_batch)
print('runtime/gene:', time_batch/len(genes))

100%|██████████| 15519/15519 [25:52<00:00, 10.00it/s] 


time batch: 0:25:52.527398
runtime/gene: 0:00:00.100040


In [29]:
df_report_complete

,reference_mesh,starting_pmidmesh,starting_pmid,starting_mesh,starting_rsid,matching_pmidmesh,matching_pmids,matching_mesh,matching_rsid,dropped_rsid,matching_mesh_ratio,matching_pmids_ratio,matching_pmidmesh_ratio,matching_rsid_ratio,matching_rsid_pmid10,matching_rsid_pmid100,matching_top10mesh,matching_top10rsid
MT-ND1,510,3603,269,764,105,482,240,52,97,8,0.068,0.892,0.134,0.924,6,0,"['DNA, Mitochondrial', 'Mitochondria', 'Mitoch...","['rs199476118', 'rs41460449', 'rs397515507', '..."
MT-ND2,510,3958,292,795,123,546,267,57,112,11,0.072,0.914,0.138,0.911,9,0,"['DNA, Mitochondrial', 'Mitochondria', 'Mitoch...","['rs199476118', 'rs41460449', 'rs397515507', '..."
MT-CO1,510,2308,163,558,97,351,155,43,93,4,0.077,0.951,0.152,0.959,5,0,"['DNA, Mitochondrial', 'Mitochondria', 'Genome...","['rs28357984', 'rs1599988', 'rs3021088', 'rs19..."
MT-CO2,510,1330,94,422,67,206,84,33,63,4,0.078,0.894,0.155,0.940,1,0,"['DNA, Mitochondrial', 'Mitochondria', 'Genome...","['rs2000975', 'rs3135028', 'rs200165736', 'rs2..."
MT-ATP8,510,1802,127,518,77,293,118,41,75,2,0.079,0.929,0.163,0.974,4,0,"['DNA, Mitochondrial', 'Mitochondria', 'Mitoch...","['rs199476133', 'rs2000975', 'rs2001031', 'rs1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CRISP1,510,22,2,19,2,1,1,1,1,1,0.053,0.500,0.045,0.500,0,0,['Protein Folding'],['rs149522268']
TMC3,510,50,4,40,3,1,1,1,1,2,0.025,0.250,0.020,0.333,0,0,['Parkinson Disease'],['rs3935740']
MPIG6B,510,210,16,107,7,8,3,8,3,4,0.075,0.188,0.038,0.429,0,0,"['Alzheimer Disease', 'Brain Ischemia', 'Intra...","['rs11575848', 'rs117894946', 'rs805294']"
PDP1,510,241,19,179,13,10,7,7,5,8,0.039,0.368,0.041,0.385,0,0,"['Brain', 'Cerebellar Diseases', 'Lipid Metabo...","['rs267606938', 'rs755840386', 'rs750529031', ..."


GRPM-Survey Version 2.0

OB-BMI (full)/
time batch: 0:42:36.831920
runtime/gene: 0:00:00.164755

======================================
GRPM-Survey Version 3.0
OB-BMI (partial)
time batch: 0:00:49.547031
runtime/gene: 0:00:00.003193

OB-BMI (full)
time batch: 0:10:30.102083
runtime/gene: 0:00:00.040602\78/

In [17]:
GRPMX_report = pd.read_csv(directory+'/GRPMX_report.csv').head(20)
GRPMX_report

FileNotFoundError: [Errno 2] No such file or directory: 'grpm_surveys/grpm_survey_pcg_ng_cvd/GRPMX_report.csv'

In [ ]:
df_read = pd.read_csv(directory+'/grpmx_filtered_output.csv', index_col=0)
print('genes matching:', df_read.gene.nunique())
print('mesh matching:', df_read.mesh.nunique())
print('apply threshold in Analyzer Module')
df_read

# Check results

In [18]:
# Visualize GRPMX_report.csv
GRPMX_report = pd.read_csv(directory+'/GRPMX_report.csv', index_col=0).rename(columns={'index':'gene'})
GRPMX_report.gene.drop_duplicates().to_clipboard()
print('Genes matching:',len(GRPMX_report.gene.drop_duplicates()))

GRPMX_report[['reference_mesh', 'starting_pmidmesh', 'starting_pmid','starting_mesh','starting_rsid', 'matching_pmidmesh', 'matching_pmids', 'matching_mesh','matching_rsid', 'dropped_rsid']] = GRPMX_report[['reference_mesh', 'starting_pmidmesh', 'starting_pmid','starting_mesh','starting_rsid', 'matching_pmidmesh', 'matching_pmids', 'matching_mesh','matching_rsid', 'dropped_rsid']].astype(int)

GRPMX_report[['matching_mesh_ratio', 'matching_pmids_ratio','matching_pmidmesh_ratio', 'matching_rsid_ratio']] = GRPMX_report[['matching_mesh_ratio', 'matching_pmids_ratio','matching_pmidmesh_ratio','matching_rsid_ratio']].astype(float)

columns_to_keep = ['matching_pmids','matching_pmids_ratio','matching_mesh','matching_rsid']
GRPMX_report_less = GRPMX_report[columns_to_keep]

sorting_column = 'matching_pmids'
GRPMX_report_sort = GRPMX_report.sort_values(by=sorting_column, ascending=False)

columns_to_display = ['gene', 'matching_pmidmesh', 'matching_pmids',
                      'matching_mesh', 'matching_rsid', 'dropped_rsid', 'matching_mesh_ratio',
                      'matching_pmids_ratio', 'matching_pmidmesh_ratio',
                      'matching_rsid_ratio']
GRPMX_report_display = GRPMX_report[columns_to_display]
GRPMX_report_display

FileNotFoundError: [Errno 2] No such file or directory: 'grpm_surveys/grpm_survey_pcg_ng_cvd/GRPMX_report.csv'

In [ ]:
# Matching PMIDs in Database
GRPMX_report_sort = GRPMX_report.sort_values(by= 'matching_pmids',ascending=False)

x = GRPMX_report_sort.gene.iloc[:40]
y = GRPMX_report_sort['matching_pmids'].iloc[:40]
plt.figure(figsize=(5, len(x)*0.2))
plt.title('Matching PMIDs in Database', loc='center',pad=10)

plt.barh(x,y)
plt.gca().invert_yaxis()
plt.tick_params(axis='x', which='both', top=True, bottom=False, labeltop=True, labelbottom=False)
#plt.xlabel('pmid count', position=(0.5, 1.08))
plt.ylabel('genes')
plt.xlabel('matching pmid', position=(0.5, 1.08))
ax = plt.gca()
ax.xaxis.set_label_position('top')

plt.show()

In [ ]:
# Add "interest value" to report:----------------------------------------------------------
max_match_pmids = int(GRPMX_report['matching_pmids'].max())
GRPMX_report_int = GRPMX_report
GRPMX_report_int['matching_pmids_score'] = round((GRPMX_report_int['matching_pmids']/max_match_pmids),3)

GRPMX_report_int['interest_value'] = round(GRPMX_report_int['matching_pmids_score'] * GRPMX_report_int['matching_pmids_ratio'],3)

GRPMX_report_int.set_index('gene').sort_values(by='interest_value')#.T

In [ ]:
# Matching PMIDs in Database
GRPMX_report_sort = GRPMX_report.sort_values(by= 'matching_pmids_index',ascending=False)

x = GRPMX_report_sort.gene.iloc[:100]
y = GRPMX_report_sort['matching_pmids_index'].iloc[:100]
plt.figure(figsize=(5, len(x)*0.2))
plt.title('Matching PMIDs in Database', loc='center',pad=10)

plt.barh(x,y)
plt.gca().invert_yaxis()
plt.tick_params(axis='x', which='both', top=True, bottom=False, labeltop=True, labelbottom=False)
#plt.xlabel('pmid count', position=(0.5, 1.08))
plt.ylabel('genes')
plt.xlabel('matching pmid', position=(0.5, 1.08))
ax = plt.gca()
ax.xaxis.set_label_position('top')

plt.show()

# Extra

Simple GRPM Subsetting

In [ ]:
grpm_dataset.head()

## filter by mesh

In [ ]:
# filtering source dataset
import time
timea = time.time()

my_mesh = 'Heart Failure'
#nbib_subset = pd.DataFrame(columns= nbib_dataset.columns)
filteres_grpm = grpm_dataset[grpm_dataset.mesh == my_mesh].reset_index(drop=True)

print((time.time()-timea)/60,'minutes')
filteres_grpm.to_csv('filteres_grpm_heart_fail.csv') #= pd.read_csv(filteres_grpm)
filteres_grpm

In [ ]:
# import LitVat-PubMed Dataset (GRPM)
filteres_grpm = pd.read_csv('filteres_grpm_heart_fail.csv', index_col=0)
filteres_grpm.pmid = filteres_grpm.pmid.astype('str')  # convert PMIDs to str
filteres_grpm

In [ ]:
#Analyze data with "groupby.describe" method

## 1. groupby.describe analysis by [pmids]
filteres_grpm_gene = filteres_grpm.groupby('gene').describe().reset_index()#.reset_index(drop=True)
filteres_grpm_gene[['gene','pmid']].sort_values(by=('pmid', 'unique'), ascending=False).reset_index(drop=True)

## filter by gene

In [ ]:
# filtering source dataset
import time
timea = time.time()
my_mesh = ref.mesh[0]
my_gene = 'GLA'
#nbib_subset = pd.DataFrame(columns= nbib_dataset.columns)
filteres_grpm = grpm_dataset[grpm_dataset.gene == my_gene].reset_index(drop=True)
print((time.time()-timea)/60,'minutes')

filteres_grpm.to_csv('filteres_grpm.csv') #= pd.read_csv(filteres_grpm)

In [ ]:
# import LitVat-PubMed Dataset (GRPM)
filteres_grpm = pd.read_csv('filteres_grpm.csv', index_col=0)
filteres_grpm_sub = filteres_grpm[filteres_grpm.mesh.str.contains('Fabry')].reset_index(drop=True)
filteres_grpm_sub.pmid = filteres_grpm.pmid.astype('str')  # convert PMIDs to str
filteres_grpm_sub

In [ ]:
#Analyze data with "groupby.describe" method

## 1. groupby.describe analysis by [pmids]
filteres_grpm_sub_pmids = filteres_grpm.groupby('pmid').describe().reset_index()
filteres_grpm_sub_pmids

In [ ]:
## 1. groupby.describe analysis by [rsid]
filteres_grpm_sub_rsid = filteres_grpm.groupby('rsid').describe().reset_index()
filteres_grpm_sub_rsid

In [ ]:
## 1. groupby.describe analysis by [rsid]
filteres_grpm_mesh = filteres_grpm.groupby('mesh').describe().reset_index()
filteres_grpm_mesh

In [ ]:
filteres_grpm[filteres_grpm.mesh.str.contains('Fabry')]

In [ ]:

filteres_grpm.columns = filteres_grpm.columns.to_flat_index()
#new_column_names = ['rsid', 'pmid-count', 'pmid-unique','pmid-top','pmid-freq','mesh-count', 'mesh-unique','mesh-top','mesh-freq']
filteres_grpm.columns = filteres_grpm
#------------------
filteres_grpm

## report transposer

In [0]:
def report_transposer(path):
    #pd.read_csv(directory +'/GRPMX_report.csv', index_col=0)
    GRPMX_report = pd.read_csv(path, index_col=0).transpose().reset_index()
    GRPMX_report.to_csv(path)

path = r"..\grpm_surveys\grpm_survey_pcg_cvd\GRPMX_report.csv"
report_transposer(path)

In [0]:
for i in ['norep1','norep2','norep3','norep4','norep5','norep6','norep7','norep8','norep9','norep10','norep11']:
    path = rf"..\grpm_surveys\grpm_random\grpm_random_pcg_{i}\GRPMX_report.csv"
    report_transposer(path)

## SQL Query [alternative]

In [16]:
%%time
!pip install pandasql
import pandas as pd
import pandasql as ps

ref_mesh_tuple = tuple(ref_mesh_list)
grpm_dataset_ = grpm_dataset[:len(grpm_dataset)//1]
"""
Complete SQL Query time:
CPU times: total: 3min 40s
Wall time: 4min 16s
"""

# Define SQL query
query = f"""
SELECT DISTINCT * FROM grpm_dataset_
WHERE mesh IN {ref_mesh_tuple}
"""

# Run SQL query
grpm_match_full = ps.sqldf(query, locals())
grpm_match_full[['gene', 'rsid', 'pmid', 'mesh']].head()

PendingRollbackError: Can't reconnect until invalid transaction is rolled back.  Please rollback() fully before proceeding (Background on this error at: https://sqlalche.me/e/20/8s2b)